# Mock Analysis

This is a mock analysis of the ASUU Strike Project. This is to help us figure out what to do with the data and optimise the process.

### Import packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

### Load data into dataframe

In [ ]:
sample_df = pd.read_csv("sample.csv")

sample_df.head()

In [ ]:
sample_df.info()

## Clean data

Drop all non-UNILAG students

In [ ]:
sample_df = sample_df[sample_df["Are you a student of UNILAG?"] == "Yes"]

Drop unrequired columns

In [ ]:
cols_to_drop = ["Timestamp", 
                "Are you a student of UNILAG?",
                "How has the just concluded strike affected you ?",
                "What challenges did you experience during resumption after the long strike?",
                "Would you like to be contacted for follow-up questions? If so, kindly drop your email.\nOnly drop your email if you're comfortable doing so!",
                ]
sample_df.drop(columns=cols_to_drop, axis=1, inplace=True)
sample_df.info()

Rename columns

In [ ]:
new_col_names = [
    "level", "age", "gender", "marital_status", "faculty", "department", "skill", "prep_before", "prep_after", 
    "quality_before", "quality_after", "confidence_before", "confidence_after", 
    "cgpa_before", "cgpa_after", "employment", "quit_job"
]

sample_df.columns = new_col_names

sample_df.head()

**Remove Outliers**

Ouliers include all 100 Level students, as they have nothing to compare against. Some students might have entered wrong values for CGPA. Take note of that as well.

In [ ]:
# remove 100 level students
sample_df = sample_df[sample_df["level"]!="100level"]

# remove outliers (CGPA = 0)
sample_df = sample_df[
    (sample_df["cgpa_before"]>0) & (sample_df["cgpa_after"]>0)
    ]
sample_df.describe()

Compute target variable

In [ ]:
sample_df["cgpa_change"] = sample_df["cgpa_after"] - sample_df["cgpa_before"]

sample_df.describe()

## Exploratory Data Analysis

## Build Model

### Baseline

### Iterate

### Evaluate

## Communicate